# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

In [91]:
df = pd.read_csv('homepage_actions.csv')

In [3]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [22]:
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [23]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [92]:
clickers = set(df[df['action']=='click']['id'])
viewers = set(df[df['action']=='view']['id'])
experiment = set(df[df['group']=='experiment']['id'])
control = set(df[df['group']=='control']['id'])
print(f'# of people who clicked: {len(clickers)}')
print(f'# of people who viewed: {len(viewers)}')
print(f'# of clickers who did not view: {len(clickers - viewers)}')
print(f'# of viwers who did not click: {len(viewers - clickers)}')
print(f'# of people in both groups: {len(control&experiment)}')

# of people who clicked: 1860
# of people who viewed: 6328
# of clickers who did not view: 0
# of viwers who did not click: 4468
# of people in both groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [93]:
df['viewed'] = 1
df['clicked'] = 0
df.loc[df['id'].isin(clickers),'clicked'] = 1

In [105]:
df2 = df.drop_duplicates(subset=['id'])[['group','viewed','clicked']]

In [119]:
table = df2.groupby('group').sum()
control = table.iloc[0]
experiment = table.iloc[1]

In [122]:
stats.chisquare(f_obs=experiment, f_exp=control)

Power_divergenceResult(statistic=33.89952032315072, pvalue=5.8033006040559e-09)

In [125]:
exp = df2[df2['group']=='experiment']['clicked']
con = df2[df2['group']=='control']['clicked']
stats.ttest_ind(exp, con, equal_var=False)

Ttest_indResult(statistic=2.615440020788211, pvalue=0.008932805628674203)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [130]:
control_rate = control[1]/control[0]
expected = experiment[0]*control_rate

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [132]:
table

,viewed,clicked
group,,
control,3332,932
experiment,2996,928


In [143]:
n = experiment[0]
p = control_rate
var = n*p*(1-p)

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [144]:
np.sqrt(var)

24.568547907005815

In [145]:
actual = experiment[1]
expected

z = (actual - expected)/np.sqrt(var)

In [146]:
stats.norm.sf(z)

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yes, although using the chi-square test resulted in a much smaller p-value than finding the z-score, in all three cases we would reject the null hypothesis which indicates that there is a significant difference between the two websites. 

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.